In [1]:
import h5py
import keras
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.layers import LSTM, CuDNNLSTM
from keras.optimizers import Adam
from keras.layers import Activation
import tensorflow as tf
import pandas as pd

import matplotlib.pyplot as plt




## Load Data from file

In [2]:
# def load_hdf_dataset(file_path, h5_key, input_key, output_key):
#     df = pd.read_hdf(file_path, h5_key)
#     x = np.array(df[[input_key]].to_numpy())
#     y = np.array(df[[output_key]].to_numpy())
#     x = np.array([np.array(row[0]) for row in x])
#     y = np.array([np.array(row[0]) for row in y])
#     return x, y
# h5_key = 'ib-samples'
# dataset_input_key = 'merged_input'
# dataset_output_key = 'merged_output'
# dataset_file_path = 'tmp/ib-out/ib-samples.h5'

def load_hdf_dataset(file_path, h5_key, input_key, output_key):
    #hf = h5py.File(file_path, 'r')
    dataset = np.load(file_path, allow_pickle=True)
    print(type(dataset))
    x = dataset[()][input_key]
    #x = np.array([np.array(row[0]) for row in x])

    y = dataset[()][output_key]
    #y = np.array([np.array(row[0]) for row in y])

    #hf.close()
    return x, y
h5_key = 'ib-samples'
dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples.npy'

x, y = load_hdf_dataset(file_path=dataset_file_path, h5_key=h5_key, input_key=dataset_input_key, output_key= dataset_output_key)


<class 'numpy.ndarray'>


In [3]:
from sys import getsizeof

x.shape

(969000, 30, 9)

## Create Dataset

In [4]:

timesteps, data_dim = np.shape(x[0])


In [5]:
batch_size = 3230

def create_model():

    dynamic_model_input_layer = Input(shape=(timesteps, data_dim), batch_size=batch_size)
    dynamic_model_lstm_layer = LSTM(32,return_state=False,return_sequences=True, activation="sigmoid",stateful=True, dropout=0.7)(dynamic_model_input_layer)
    dynamic_model_dense_layer = Dense(y.shape[2],activation="relu")(dynamic_model_lstm_layer)
    model = Model(inputs=dynamic_model_input_layer, outputs=dynamic_model_dense_layer)
    model.compile(loss=['mean_squared_error'] , optimizer=Adam(0.001))
    #model.build()
    #model.summary()
    return model
model = create_model()

2022-08-11 11:15:31.086640: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 11:15:31.114646: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 11:15:31.114829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 11:15:31.115302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
def scheduler(epoch, lr):
  if epoch == 200:
    return lr*0.1
  else:
      return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history = model.fit(x, y, epochs=1000, batch_size=batch_size, validation_split=0.3, shuffle=False, callbacks=[lr_callback])


2022-08-11 11:15:31.858597: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 732564000 exceeds 10% of free system memory.
2022-08-11 11:15:32.259192: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 732564000 exceeds 10% of free system memory.


Epoch 1/1000
 13/210 [>.............................] - ETA: 2s - loss: 13443.9844

2022-08-11 11:15:33.745210: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


210/210 [==============================] - 5s 17ms/step - loss: 12597.2334 - val_loss: 12379.7217 - lr: 0.0010
Epoch 2/1000
210/210 [==============================] - 3s 15ms/step - loss: 11421.1533 - val_loss: 11439.2793 - lr: 0.0010
Epoch 3/1000
210/210 [==============================] - 3s 15ms/step - loss: 10616.9668 - val_loss: 10729.0332 - lr: 0.0010
Epoch 4/1000
210/210 [==============================] - 3s 15ms/step - loss: 9982.7549 - val_loss: 10136.8330 - lr: 0.0010
Epoch 5/1000
210/210 [==============================] - 3s 15ms/step - loss: 9446.3838 - val_loss: 9628.1572 - lr: 0.0010
Epoch 6/1000
210/210 [==============================] - 3s 15ms/step - loss: 8983.6250 - val_loss: 9185.0625 - lr: 0.0010
Epoch 7/1000
210/210 [==============================] - 3s 15ms/step - loss: 8579.7051 - val_loss: 8795.0850 - lr: 0.0010
Epoch 8/1000
210/210 [==============================] - 3s 16ms/step - loss: 8223.6934 - val_loss: 8448.5430 - lr: 0.0010
Epoch 9/1000
210/210 [========

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model_path = 'dynamic_model/model.h5'
model.save(model_path)

In [ ]:
del model

In [ ]:
model = keras.models.load_model(model_path)

In [ ]:
x[0].shape

In [ ]:
model_out = model.predict(np.array( [x[0],]))

In [ ]:
model_out

In [ ]:
y[0]